In [1]:
%pip install python-dotenv

You should consider upgrading via the '/opt/appimages/cursor.AppImage -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os

api_key = os.getenv("GOOGLE_API_KEY")

In [9]:
import base64
import requests
import json

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

image_path = "/home/luis/workspace/vacocam_render/experiments/vacocam/2024_05_03-4_gemini1.0_3sec/1_original/track/presentation/prerender.mp4_05-33_05-36.png"
metadata_path = "/home/luis/workspace/vacocam_render/experiments/vacocam/2024_05_03-4_gemini1.0_3sec/1_original/track/presentation/prerender.mp4_05-33_05-36.txt"
metadata = open(metadata_path, "r").read()

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
}

text = f"""
Look at this still from 3 seconds of football video. 
The colored area(s) with a letter label are clusters of ball detections across multiple frames.

The areas metadata is the following:

{metadata}

Your task is to determine which areas are ball detections belonging to the primary match, using their positioning, context, trajectory, time, etc.
There may or not be more than one match being played. Also, an object that is not a ball may have been labeled as a ball.

Pay close attention to the lines, goals and other markers to determine the bounds and action of the primary match.
The drawn arrows indicate the general direction of the detections.

You only need your visual understanding of the image as-is to accomplish this task, you dont need any additional information.

Respond with a top-level array of objects with keys "id": String, "primary": Boolean, "reason" String ["primary ball", "irrelevant ball", "not ball"].
Respond with valid JSON stripped of any display formatting (including blockquotes, etc.), comments, or explanations
Respond with labels ordered from most to least relevant to the primary match.
notalk; justgo
"""

payload = {
  "contents": [
    {
      "parts": [
        {
          "text": "input: "
        },
        {
          "inlineData": {
            "mimeType": "image/png",
            "data": base64_image
          }
        },
        {
          "text": " Look at this still from 3 seconds of football video.\nThe colored area(s) with a letter label are clusters of ball detections across multiple frames.\n\nThe areas metadata is:\nA, Red\nB, Cyan\n\nYour task is to determine which areas are ball detections belonging to the primary match, using their positioning, context, trajectory, time, etc.\nThere may or not be more than one match being played. Also, an object that is not a ball may have been labeled as a ball.\nPay close attention to the lines, goals and other markers to determine the bounds and action of the primary match.\nThe drawn arrows indicate the general direction of the detections.\nYou only need your visual understanding of the image as-is to accomplish this task, you dont need any additional information.\nRespond with a top-level array of objects with keys \"id\": String, \"primary\": Boolean, \"reason\" String [\"primary ball\", \"irrelevant ball\", \"not ball\"].\nRespond with valid JSON stripped of any display formatting (including blockquotes, etc.), comments, or explanations\nRespond with labels ordered from most to least relevant to the primary match.\nnotalk; justgo"
        }
      ]
    },
  ],
  "generationConfig": {
    "maxOutputTokens": 512,
    "temperature": 0.1,
  }
}

print(str(text))

response = requests.post(f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro-vision-latest:generateContent?key={api_key}", headers=headers, json=payload)

print(json.dumps(response.json(), indent=2))


Look at this still from 3 seconds of football video. 
The colored area(s) with a letter label are clusters of ball detections across multiple frames.

The areas metadata is the following:

id, color
A, Blue
B, Pumpkin

Your task is to determine which areas are ball detections belonging to the primary match, using their positioning, context, trajectory, time, etc.
There may or not be more than one match being played. Also, an object that is not a ball may have been labeled as a ball.

Pay close attention to the lines, goals and other markers to determine the bounds and action of the primary match.
The drawn arrows indicate the general direction of the detections.

You only need your visual understanding of the image as-is to accomplish this task, you dont need any additional information.

Respond with a top-level array of objects with keys "id": String, "primary": Boolean, "reason" String ["primary ball", "irrelevant ball", "not ball"].
Respond with valid JSON stripped of any display f